# Exercises

These small exercises are intended to build on the knowledge that you acquired in the [first](TV_denoising_with_PDHG.ipynb) [two](Using_a_different_solver.ipynb) notebooks. The exercises do not depend on each other. Feel free to pick those you like and do them in any order.

## General note

The most important technique to solve the exercises is to browse the [ODL examples](https://github.com/odlgroup/odl/tree/master/examples) and "steal" code snippets from there. You will need to make adaptions, of course, but in general you can find a lot of things already done and explained in a comment. 

If you have trouble understanding the explanations, or if there is an error, please [let us know](https://github.com/odlgroup/odl/issues). These examples are meant to be understandable pretty much without prior knowledge, and we appreciate any feedback on them.

## 1. Tomography

Instead of denoising, solve the TV-regularized tomography problem

$$
    \min_{x \geq 0} \left[ \frac{1}{2} \| A x - y \|_2^2 + \alpha \| \nabla x \|_1 \right],
$$

where $A$ is the ray transform. You may pick your favorite acquisition geometry (2D, 3D, paralell beam, cone beam, ...).

### Good to know

- Find out how to set up the ray transform as ODL operator. You should use the ASTRA toolbox as backend if possible.
- The data fit $\|\cdot -y\|_2^2$ is now defined on a different space than before, and also $y$ is no longer an element of the reconstruction space $X$. Make sure you understand the details.

## 2. Smooth solvers

ODL also has a number of numerical solvers for smooth problems, most of them first-order methods, i.e., methods that involve the gradient of the cost function. The [LBFGS](https://github.com/odlgroup/odl/blob/ad32a286b69f34260d4428d7282b4058ed2e2603/odl/solvers/smooth/newton.py#L247-L487) method is the limited memory variant of the popular [BFGS algorithm](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm) for smooth optimization.

Use it to solve the smoothed variant

$$
    \min_{x} \left[ \frac{1}{2} \| x - y \|_2^2 + \alpha \| \nabla x \|_{1, \epsilon} \right]
$$

of the TV denoising problem, where $\|\cdot\|_{1, \epsilon}$ is the [Huber function](https://en.wikipedia.org/wiki/Huber_loss)

$$
    \|x\|_{1, \epsilon} = \int \left|x(t)\right|_\epsilon\, \mathrm{d} t, \\
    |u|_\epsilon =
    \begin{cases}
        \frac{1}{2} |u|_2^2,                 & \text{if } |u| \leq \epsilon, \\
        \epsilon |u| - \frac{\epsilon^2}{2}, & \text{if } |u| > \epsilon.
    \end{cases}
$$

It is a smoothed variant of the 1-norm that is differentiable everywhere.

### Good to know

- You may think that this is a lot of work. It isn't! That's because the Huber function is the **Moreau envelope** of the 1-norm. You don't need to know what it is exactly -- the important property that you will apply is

  $$
      \nabla \|\cdot\|_{1, \epsilon}(x) = \frac{1}{\epsilon}\left( x - \mathrm{prox}_{\epsilon \|.\|_1}(x) \right).
  $$
  
  Conveniently, there is the [`odl.solvers.MoreauEnvelope`](https://github.com/odlgroup/odl/blob/ad32a286b69f34260d4428d7282b4058ed2e2603/odl/solvers/functional/default_functionals.py#L2204-L2292) class that implements this property for you.
  
